In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# matplotlib 폰트설정
# plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rc('font', family='Malgun Gothic') # For Windows
print(plt.rcParams['font.family'])

%matplotlib inline
# 브라우저에서 바로 이미지를 그린다.v

['Malgun Gothic']


In [3]:
# 지수표현 제거
pd.options.display.float_format = '{:.5f}'.format

# 1. User데이터 전처리 작업
- 군집화 하기 위해 필요한 변수 선정
- 한 데이터프레임에 하나의 suer_id만 들어가도록 작업

In [3]:
user = pd.read_csv('data/except_personal_rehailitation.csv')
user.head()

,Unnamed: 0,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,...,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt,gender,rehabilitation_yn,personal_rehabilitation_yn,personal_rehabilitation_complete_yn
0,0,1748340,2022-06-07 13:05:41,7,191,42000000.00000,13.60000,NaN,430982.00000,1996.00000,...,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000,0,0.00000,0.00000
1,1,1748340,2022-06-07 13:05:41,25,169,24000000.00000,17.90000,NaN,430982.00000,1996.00000,...,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000,0,0.00000,0.00000
2,2,1748340,2022-06-07 13:05:41,2,7,24000000.00000,18.50000,NaN,430982.00000,1996.00000,...,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000,0,0.00000,0.00000
3,3,1748340,2022-06-07 13:05:41,4,268,29000000.00000,10.80000,NaN,430982.00000,1996.00000,...,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000,0,0.00000,0.00000
4,4,1748340,2022-06-07 13:05:41,11,118,5000000.00000,16.40000,NaN,430982.00000,1996.00000,...,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000,0,0.00000,0.00000


In [4]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13442614 entries, 0 to 13442613
Data columns (total 25 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   Unnamed: 0                           int64  
 1   application_id                       int64  
 2   loanapply_insert_time                object 
 3   bank_id                              int64  
 4   product_id                           int64  
 5   loan_limit                           float64
 6   loan_rate                            float64
 7   is_applied                           float64
 8   user_id                              float64
 9   birth_year                           float64
 10  insert_time                          object 
 11  credit_score                         float64
 12  yearly_income                        float64
 13  income_type                          object 
 14  company_enter_month                  float64
 15  employment_type               

In [5]:
# 사용할 column만 선택
used_cols = [
    'user_id',
    'loanapply_insert_time',
    'bank_id',
    'product_id',
    'loan_limit',
    'loan_rate',
    'birth_year',
    'credit_score',
    'yearly_income',
    'income_type',
    'employment_type',
    'houseown_type',
    'desired_amount',
    'purpose',
    'existing_loan_cnt',
    'existing_loan_amt',
    'gender',
    
]

In [6]:
user_for_cluster = user[used_cols]
user_for_cluster.head()

,user_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,birth_year,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt,gender
0,430982.00000,2022-06-07 13:05:41,7,191,42000000.00000,13.60000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
1,430982.00000,2022-06-07 13:05:41,25,169,24000000.00000,17.90000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
2,430982.00000,2022-06-07 13:05:41,2,7,24000000.00000,18.50000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
3,430982.00000,2022-06-07 13:05:41,4,268,29000000.00000,10.80000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
4,430982.00000,2022-06-07 13:05:41,11,118,5000000.00000,16.40000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000


In [7]:
#전처리 작업
## 병합할 수 있게 loanapply_insert_time => insert_date
user_for_cluster['loanapply_insert_time'] = user_for_cluster['loanapply_insert_time'].apply(lambda x: x[:10])
user_for_cluster

C:\Anaconda\envs\bigcon\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,birth_year,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt,gender
0,430982.00000,2022-06-07,7,191,42000000.00000,13.60000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
1,430982.00000,2022-06-07,25,169,24000000.00000,17.90000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
2,430982.00000,2022-06-07,2,7,24000000.00000,18.50000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
3,430982.00000,2022-06-07,4,268,29000000.00000,10.80000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
4,430982.00000,2022-06-07,11,118,5000000.00000,16.40000,1996.00000,620.00000,24000000.00000,EARNEDINCOME,정규직,자가,25000000.00000,대환대출,2.00000,15000000.00000,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13442609,478802.00000,2022-06-03,62,200,3000000.00000,14.80000,1965.00000,610.00000,45000000.00000,EARNEDINCOME,정규직,기타가족소유,50000000.00000,대환대출,2.00000,56000000.00000,1.00000
13442610,478802.00000,2022-06-03,2,7,40000000.00000,11.80000,1965.00000,610.00000,45000000.00000,EARNEDINCOME,정규직,기타가족소유,50000000.00000,대환대출,2.00000,56000000.00000,1.00000
13442611,478802.00000,2022-06-03,32,257,15000000.00000,7.20000,1965.00000,610.00000,45000000.00000,EARNEDINCOME,정규직,기타가족소유,50000000.00000,대환대출,2.00000,56000000.00000,1.00000
13442612,478802.00000,2022-06-03,33,110,44000000.00000,13.50000,1965.00000,610.00000,45000000.00000,EARNEDINCOME,정규직,기타가족소유,50000000.00000,대환대출,2.00000,56000000.00000,1.00000


In [9]:
# 나이대로 변환(ex.20, 30)
user_for_cluster['birth_year'] = user_for_cluster['birth_year'].apply(lambda x: int((2022 - x)/10) * 10)
user_for_cluster['birth_year']

C:\Anaconda\envs\bigcon\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0           20
1           20
2           20
3           20
4           20
            ..
13442609    50
13442610    50
13442611    50
13442612    50
13442613    50
Name: birth_year, Length: 13442614, dtype: int64

In [10]:
# user_id 형식 int로 바꿔주기
user_for_cluster['user_id'] =user_for_cluster['user_id'].astype(int)

C:\Anaconda\envs\bigcon\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
# 고유 user_id 뽑아내기
user_id_list = user_for_cluster['user_id'].unique()
user_id_list.__len__()

314147

In [13]:
info_dict = {f'{name}_list': [] for name in user_for_cluster.columns}
list_names = list(info_dict.keys())

from tqdm import tqdm
for i in tqdm(user_id_list):
    user_row = user_for_cluster[user_for_cluster['user_id'] == i]
    #동일한 user_id 있으면 가장 마지막 user_id의 정보 선택
    information = user_row.iloc[-1]
    
    # 정보 저장
    for list_name, elem in zip(list_names, information):
        info_dict[list_name].append(elem)
    

100%|██████████| 314147/314147 [1:13:58<00:00, 70.79it/s]


In [14]:
df = pd.DataFrame(info_dict)

#column 이름 이상해서 원래대로 (ex. user_id_list -> user_id)
new_colname = {oldname: oldname[:-5] for oldname in list(info_dict.keys())}
df.rename(columns=new_colname, inplace=True)

In [17]:
# 추가적으로 안쓸 데이터 제거
df.drop(['bank_id', 'product_id', 'loan_limit', 'loan_rate'], axis=1, inplace=True)

In [7]:
df.head()

,user_id,loanapply_insert_time,birth_year,credit_score,yearly_income,income_type,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt,gender
0,430982,2022-05-16,20,620.00000,2000000.00000,EARNEDINCOME,정규직,자가,35000000.00000,대환대출,2.00000,15000000.00000,1.00000
1,345273,2022-05-21,40,700.00000,60000000.00000,EARNEDINCOME,정규직,자가,50000000.00000,생활비,3.00000,86000000.00000,1.00000
2,3058,2022-06-03,30,910.00000,44000000.00000,FREELANCER,기타,전월세,22000000.00000,생활비,3.00000,23000000.00000,1.00000
3,181137,2022-06-04,20,740.00000,30000000.00000,EARNEDINCOME,정규직,전월세,10000000.00000,전월세보증금,3.00000,43000000.00000,1.00000
4,197454,2022-06-07,30,550.00000,41000000.00000,EARNEDINCOME,계약직,전월세,5000000.00000,전월세보증금,2.00000,42000000.00000,1.00000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314147 entries, 0 to 314146
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                314147 non-null  int64  
 1   loanapply_insert_time  314147 non-null  object 
 2   birth_year             314147 non-null  int64  
 3   credit_score           314147 non-null  float64
 4   yearly_income          314147 non-null  float64
 5   income_type            314147 non-null  object 
 6   employment_type        314147 non-null  object 
 7   houseown_type          314147 non-null  object 
 8   desired_amount         314147 non-null  float64
 9   purpose                314147 non-null  object 
 10  existing_loan_cnt      314147 non-null  float64
 11  existing_loan_amt      314147 non-null  float64
 12  gender                 314147 non-null  float64
dtypes: float64(6), int64(2), object(5)
memory usage: 31.2+ MB


In [19]:
df.to_csv('user_cluster.final.csv', index=False)